# Fits

In [1]:
import numpy as np
from lmfit import Model

Fit functions:
- for the background, the following parametric function will be used
$$exp \left( 2 \cdot \frac{|x - a+ib|^2}{|x - c+id|^2} + f \cdot (x - c) \right)$$
- for the signal, we will use either the gaussian function
$$\mu \cdot exp \left( -\frac{1}{2} \frac{(x - x_0)^2}{\sigma^2} \right)$$
or a Maxwell-Boltzmann distribution
$$\mu \cdot \frac{x^2}{\sigma^3} \cdot exp \left( -\frac{1}{2} \frac{(x - x_0)^2}{\sigma^2} \right)$$
depending on the theoretical model we assumed.

The total shape will be given by the sum of the two functions.

In [ ]:
def bkg(x,a,b,c,d,e,f):                                                   
    return e**2*abs(x-a+1j*b)**2/abs(x-c+1j*d)**2+f*(x-c)

def gaussian(x,x0,s,mu):
    return mu * np.exp(-.5*((x-x0)/s)**2)
    
def maxwell(x,x0,s,mu):
    return mu * x**2/s**3 * np.exp(-.5*((x-x0)/s)**2)
    
    
def signal_gauss(x,a,b,c,d,e,f,x0,s,mu):
    return bkg(x,a,b,c,d,e,f) + gaussian(x,x0,s,mu)
    
def signal_maxwell(x,a,b,c,d,e,f,x0,s,mu):
    return bkg(x,a,b,c,d,e,f) + maxwell(x,x0,s,mu)

## Background

Parameters $a$ and $c$ will be initialized to the cavity frequency, 

In [ ]:
def fit_bkg(freq, fft, weights, center, ref):
    # set fit model
    bkg_model = Model(bkg)
    ps = bkg_model.make_params(a={'value':center, 'min':center*0.999, 'max':center*1.01},
                               b=2e4,
                               c={'value':center, 'min':center*0.999, 'max':center*1.01},
                               d=2.2e4,
                               e=1e-2*np.sqrt(ref),
                               f=1e-12*ref)
    # execute fit
    result = bkg_model.fit(fft,x=freq,params=ps,weights=1/weights)
    
    return(result)